In [4]:
!pip install joblib

  Obtaining dependency information for joblib from https://files.pythonhosted.org/packages/1e/e8/685f47e0d754320684db4425a0967f7d3fa70126bffd76110b7009a0090f/joblib-1.5.2-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/308.4 kB ? eta -:--:--
   - -------------------------------------- 10.2/308.4 kB ? eta -:--:--
   --- ----------------------------------- 30.7/308.4 kB 330.3 kB/s eta 0:00:01
   ---------- ---------------------------- 81.9/308.4 kB 657.6 kB/s eta 0:00:01
   ------------------------------- -------- 245.8/308.4 kB 1.5 MB/s eta 0:00:01
   ------------------------------- -------- 245.8/308.4 kB 1.5 MB/s eta 0:00:01
   ------------------------------- -------- 245.8/308.4 kB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 308.4/308.4 kB 1.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2
import mediapipe as mp
import numpy as np
from ultralytics import YOLO
import time
import joblib

# -------------------- Load models --------------------
PHONE_CLASSES = ["cell phone", "mobile phone"]
phone_model = YOLO("yolov8n.pt")

face_mesh = mp.solutions.face_mesh.FaceMesh(refine_landmarks=True, max_num_faces=1, min_detection_confidence=0.5)

POSE_LANDMARKS = [1, 33, 263, 61, 291, 199]
LEFT_EYE = [33, 133, 159, 145]
LEFT_IRIS = [468, 469, 470, 471]
RIGHT_EYE = [362, 263, 386, 374]
RIGHT_IRIS = [473, 474, 475, 476]

clf = joblib.load("gaze_model.pkl")  # Your trained classifier

# -------------------- Smoothers --------------------
class TwoDimSmoother:
    def __init__(self, alpha=0.3):
        self.alpha = alpha
        self.x = None
        self.y = None

    def apply(self, newx, newy):
        if self.x is None:
            self.x, self.y = newx, newy
        else:
            self.x = (1 - self.alpha) * self.x + self.alpha * newx
            self.y = (1 - self.alpha) * self.y + self.alpha * newy
        return int(self.x), int(self.y)

class HeadPoseSmoother:
    def __init__(self, alpha=0.2):
        self.alpha = alpha
        self.pitch = None
        self.yaw = None
        self.roll = None

    def apply(self, pitch, yaw, roll):
        if self.pitch is None:
            self.pitch, self.yaw, self.roll = pitch, yaw, roll
        else:
            self.pitch = (1 - self.alpha) * self.pitch + self.alpha * pitch
            self.yaw   = (1 - self.alpha) * self.yaw   + self.alpha * yaw
            self.roll  = (1 - self.alpha) * self.roll  + self.alpha * roll
        return self.pitch, self.yaw, self.roll

kalman_left = TwoDimSmoother()
kalman_right = TwoDimSmoother()
head_smoother = HeadPoseSmoother()

# -------------------- Eye direction --------------------
def get_eye_direction(landmarks, eye_idx, iris_idx, w, h, smoother, draw_frame=None):
    eye = [(int(landmarks[i].x * w), int(landmarks[i].y * h)) for i in eye_idx]
    iris_pts = [(int(landmarks[i].x * w), int(landmarks[i].y * h)) for i in iris_idx]
    iris_cx = sum([p[0] for p in iris_pts]) / len(iris_pts)
    iris_cy = sum([p[1] for p in iris_pts]) / len(iris_pts)
    iris_x, iris_y = smoother.apply(iris_cx, iris_cy)

    left, right = min(eye[0][0], eye[1][0]), max(eye[0][0], eye[1][0])
    top, bottom = min(eye[2][1], eye[3][1]), max(eye[2][1], eye[3][1])

    margin_x = int((right - left) * 0.15)  # smaller for sensitivity
    margin_y = int((bottom - top) * 0.20)

    # Draw CENTER rectangle
    if draw_frame is not None:
        center_left = left + margin_x
        center_right = right - margin_x
        center_top = top + margin_y
        center_bottom = bottom - margin_y
        cv2.rectangle(draw_frame, (center_left, center_top), (center_right, center_bottom), (0, 255, 0), 1)
        cv2.circle(draw_frame, (iris_x, iris_y), 2, (0, 0, 255), -1)

    # Determine gaze direction
    if iris_x < left + margin_x:
        return "LEFT", iris_x - (left + margin_x), iris_y - (top + margin_y)
    elif iris_x > right - margin_x:
        return "RIGHT", iris_x - (left + margin_x), iris_y - (top + margin_y)
    elif iris_y < top + margin_y:
        return "UP", iris_x - (left + margin_x), iris_y - (top + margin_y)
    elif iris_y > bottom - margin_y:
        return "DOWN", iris_x - (left + margin_x), iris_y - (top + margin_y)
    else:
        return "CENTER", iris_x - (left + margin_x), iris_y - (top + margin_y)

# -------------------- Head pose --------------------
def estimate_head_pose(landmarks, w, h):
    image_points = np.array([(landmarks[idx].x * w, landmarks[idx].y * h) for idx in POSE_LANDMARKS], dtype="double")
    model_points = np.array([
        (0.0, 0.0, 0.0),
        (-30.0, 0.0, -30.0),
        (30.0, 0.0, -30.0),
        (-30.0, 0.0, -90.0),
        (30.0, 0.0, -90.0),
        (0.0, 40.0, -50.0)
    ])
    focal_length = w
    center = (w / 2, h / 2)
    camera_matrix = np.array([[focal_length,0,center[0]],[0,focal_length,center[1]],[0,0,1]])
    dist_coeffs = np.zeros((4,1))
    success, rotation_vector, _ = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE)
    return rotation_vector if success else None

# -------------------- Main loop --------------------
cap = cv2.VideoCapture(0)
AWAY_THRESHOLD = 0.8
away_start_time = None

while True:
    ret, frame = cap.read()
    if not ret: break
    frame = cv2.flip(frame,1)
    h,w,_ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    warning = ""
    eyes_away = False
    phone_detected = 0

    # Face + eyes
    face_results = face_mesh.process(rgb)
    iris_offsets = (0,0,0,0)
    if face_results.multi_face_landmarks:
        landmarks = face_results.multi_face_landmarks[0].landmark

        left_dir, left_dx, left_dy = get_eye_direction(landmarks, LEFT_EYE, LEFT_IRIS, w, h, kalman_left, frame)
        right_dir, right_dx, right_dy = get_eye_direction(landmarks, RIGHT_EYE, RIGHT_IRIS, w, h, kalman_right, frame)

        rot_vec = estimate_head_pose(landmarks, w, h)
        pitch = yaw = roll = 0
        if rot_vec is not None:
            pitch, yaw, roll = head_smoother.apply(*rot_vec.ravel())

        if left_dir!="CENTER" or right_dir!="CENTER":
            eyes_away = True

    # Phone detection
    results = phone_model(frame, verbose=False)
    for box in results[0].boxes:
        cls = int(box.cls[0])
        label = phone_model.names[cls]
        conf = float(box.conf[0])
        if label in PHONE_CLASSES and conf>0.5:
            x1,y1,x2,y2 = map(int, box.xyxy[0])
            cv2.rectangle(frame,(x1,y1),(x2,y2),(0,0,255),2)
            cv2.putText(frame,"Mobile Detected ⚠️",(x1,y1-10),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)
            phone_detected = 1
            warning="⚠️ Mobile Detected!"

    # Eye-away timer
    current_time = time.time()
    eyes_away_duration = 0
    if eyes_away:
        if away_start_time is None:
            away_start_time = current_time
        eyes_away_duration = current_time - away_start_time
        if eyes_away_duration >= AWAY_THRESHOLD and warning=="":
            warning="⚠️ Not looking at screen!"
    else:
        away_start_time = None

    # -------------------- Prediction --------------------
    # Convert directions to int: LEFT=0, RIGHT=1, UP=2, DOWN=3, CENTER=4
    dir_map = {"LEFT":0,"RIGHT":1,"UP":2,"DOWN":3,"CENTER":4}
    features = [[
        dir_map[left_dir],
        dir_map[right_dir],
        left_dx, left_dy,
        right_dx, right_dy,
        pitch, yaw, roll,
        phone_detected,
        eyes_away_duration
    ]]
    pred = clf.predict(features)[0]
    prob = clf.predict_proba(features)[0,1]  # probability of distraction

    if pred==1 and warning=="":
        warning="⚠️ Predicted distraction"

    # Show probability
    cv2.putText(frame,f"Prob: {prob:.2f}",(10,h-10),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,255,0),2)

    # Show warning
    if warning:
        cv2.putText(frame,warning,(30,h-50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)

    cv2.imshow("Cheating Detection with Prediction",frame)
    if cv2.waitKey(1)&0xFF in (27,ord("q")):
        break

cap.release()
cv2.destroyAllWindows()

C:\Users\Yashi Garg\mp-env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Yashi Garg\mp-env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Yashi Garg\mp-env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Yashi Garg\mp-env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Yashi Garg\mp-env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  